Here is our graph:

S (h=10)

|-- (cost 4) -- A (h=6)

| |-- (cost 8) -- C (h=3)

| | |-- (cost 3) -- G (h=0)

| |-- (cost 2) -- D (h=4)

| |-- (cost 10) -- G (h=0)

|-- (cost 5) -- B (h=7)

|-- (cost 6) -- E (h=1)

|-- (cost 2) -- G (h=0)


As A* prioritizes the node with the lowest f(n) = g(n) + h(n), let's calculate the value of each f(n), and find the lowest one, which will be the final path found by our A* search.

So, S is our starting point, which means that g(S) = 0, and f(S) = 0 + 10 = 10.

And for the rest of the nodes we get this:

A: g = 4, h = 6, f(A) = 10

B: g = 5, h = 7, f(B) = 12

E: g = 6, h = 1, f(E) = 7

G: g = 2, h = 0, f(G) = 2

The lowest f(n) value is for G, which is equal to 2, this means that which means that the final path found by A* will be S -> G with the total cost of 2.

Greedy Best-First search will use only h(n), and ignore g, so in this case it will also choose G, as the heuristic value is 0 (and the cost will be 2 again). However, usually A* is a better way to find the optimal path, as Greedy ignores large g costs, and that way it's not guaranteed to find an optimal path.

Now let's implement the A* Search. Here is our graph dictionary:

In [1]:
graph = {
    'Yerevan': {'Gyumri': 20, 'Sevan': 60},
    'Gyumri': {'Yerevan': 20, 'Vanadzor': 40},
    'Sevan': {'Yerevan': 60, 'Dilizhan': 40},
    'Vanadzor': {'Gyumri': 40, 'Dilizhan': 50},
    'Dilizhan': {'Sevan': 40, 'Vanadzor': 50}
}

# heuristic values
heuristics = {
    'Yerevan': 90,
    'Gyumri': 85,
    'Sevan': 25,
    'Vanadzor': 35,
    'Dilizhan': 0
}

So, this was our Greedy search function:

In [2]:
def greedy(graph, start, goal, heuristics):
    front = [(heuristics[start], [start])]

    while front:
        front.sort(key=lambda x: x[0])
        h_value, path = front.pop(0)
        curr = path[-1]

        print("Currently in ", curr, ", approx. distance left: ", heuristics[curr])

        if curr == goal:
            print("Goal found")
            return path
        
        for n in graph[curr]:
            if n not in path:
                new_path = path + [n]
                front.append((heuristics[n], new_path))

    return None

In [5]:
greedy(graph, 'Yerevan', 'Dilizhan', heuristics)

Currently in  Yerevan , approx. distance left:  90
Currently in  Sevan , approx. distance left:  25
Currently in  Dilizhan , approx. distance left:  0
Goal found


['Yerevan', 'Sevan', 'Dilizhan']

Now let's modify this, to combine both g(n) and h(n) in order to create our A* function.

In [25]:
def a_star_search(graph, start, goal, heuristics):
    front = [(heuristics[start], 0, [start])]

    while front:
        front.sort(key=lambda x: x[0])
        f_value, g_value, path = front.pop(0)
        curr = path[-1]

        print("Currently in ", curr, ", distance travelled : ", g_value, ", approx. distance left: ", heuristics[curr], ", f = ", f_value)

        if curr == goal:
            print(f"Goal found! Total cost: {g_value}")
            return path
        
        for n, cost in graph[curr].items():
            if n not in path:
                new_g = g_value + cost
                new_f = new_g + heuristics[n]
                new_path = path + [n]
                front.append((new_f, new_g, new_path))

    return None

In [26]:
a_star_search(graph, 'Yerevan', 'Dilizhan', heuristics)

Currently in  Yerevan , distance travelled :  0 , approx. distance left:  90 , f =  90
Currently in  Sevan , distance travelled :  60 , approx. distance left:  25 , f =  85
Currently in  Dilizhan , distance travelled :  100 , approx. distance left:  0 , f =  100
Goal found! Total cost: 100


['Yerevan', 'Sevan', 'Dilizhan']

Now let's look at the BFS search, to see whether we get the same path or not.

In [21]:
def bfs(graph, start, goal):
    queue = [[start]]
    visited = set([start])

    while queue:
        path = queue.pop(0)
        node = path[-1]

        if node == goal:
            return path
        
        for i in graph[node]:
            if i not in visited:
                visited.add(i)
                new_path = path + [i]
                queue.append(new_path)

    return None

In [23]:
path = bfs(graph, "Yerevan", "Dilizhan")
print("The BFS path is: ", path)

The BFS path is:  ['Yerevan', 'Sevan', 'Dilizhan']


As we can see, in this case the BFS, A* and Greedy functions return the same result. In order to make the A* algorithm give us a different result as the best optimal path, we need to change the g(n) values, so let's do it.

In [24]:
new_graph = {
    'Yerevan': {'Gyumri': 50, 'Sevan': 90},
    'Gyumri': {'Yerevan': 50, 'Vanadzor': 10},
    'Sevan': {'Yerevan': 90, 'Dilizhan': 70},
    'Vanadzor': {'Gyumri': 10, 'Dilizhan': 20},
    'Dilizhan': {'Sevan': 70, 'Vanadzor': 20}
}

Now if we try the same functions that we have written for A*, Greedy and BFS searches with this new graph that has changed g(n) values, we'll se that the A* function gives us a new optimal path, while the other 2 functions give us the same result as before.

In [31]:
print("Greedy Best-First:", greedy(new_graph, 'Yerevan', 'Dilizhan', heuristics))
print("BFS:", bfs(new_graph, "Yerevan", "Dilizhan"))
print("A*:", a_star_search(new_graph, 'Yerevan', 'Dilizhan', heuristics))

Currently in  Yerevan , approx. distance left:  90
Currently in  Sevan , approx. distance left:  25
Currently in  Dilizhan , approx. distance left:  0
Goal found
Greedy Best-First: ['Yerevan', 'Sevan', 'Dilizhan']
BFS: ['Yerevan', 'Sevan', 'Dilizhan']
Currently in  Yerevan , distance travelled :  0 , approx. distance left:  90 , f =  90
Currently in  Sevan , distance travelled :  90 , approx. distance left:  25 , f =  115
Currently in  Gyumri , distance travelled :  50 , approx. distance left:  85 , f =  135
Currently in  Vanadzor , distance travelled :  60 , approx. distance left:  35 , f =  95
Currently in  Dilizhan , distance travelled :  80 , approx. distance left:  0 , f =  80
Goal found! Total cost: 80
A*: ['Yerevan', 'Gyumri', 'Vanadzor', 'Dilizhan']


As we can see, the Greedy and BFS functions return us the path ['Yerevan', 'Sevan', 'Dilizhan'], while the optimal path found by A* with these new values is ['Yerevan', 'Gyumri', 'Vanadzor', 'Dilizhan'].

Now let's run the Dijkstra algorithm function.

In [45]:
def dijkstra(graph, start, target):
    dist = {}
    prev = {}

    for v in graph:
        dist[v] = float('inf')
        prev[v] = None

    dist[start] = 0

    Q = set(graph.keys())

    while Q:
        u = None
        min_dist = float('inf')
        for vertex in Q:
            if dist[vertex] < min_dist:
                min_dist = dist[vertex]
                u = vertex

        Q.remove(u)

        for v, weight in graph[u].items():
            if v in Q:
                alt = dist[u] + weight
                if alt < dist[v]:
                    dist[v] = alt
                    prev[v] = u

    path = []
    curr = target
    while curr is not None:
        path.append(curr)
        curr = prev[curr]
    path.reverse()

    if path[0] != start:
        return float('inf'), None

    return path

In [46]:
distances = dijkstra(graph, "Yerevan", "Dilizhan")
print(f"Dijkstra optimal path is {distances}")

Dijkstra optimal path is ['Yerevan', 'Sevan', 'Dilizhan']


Though Dijkstra is also a good algorithm, it goes through all nodes, which may make it slower on large graphs. That's why A* is faster and more efficient for path finding.